# Notebook under development to import from excel and ReqIF to Capella model.

In [1]:
import capellambse
import populate_requirements_helper
import ipywidgets as widgets
from IPython.core.display import HTML
import pandas as pd

#String processing does like objects of type None. This is case when you pull content expecting to be a string and None is found.
def convert_nonetype (instr):
    if instr is None :
        instr = ""
        return instr
    else:
        return instr


resources = {
    "Trail_Power_for_Campgrounds": "Trail_Power_for_Campgrounds/Trail Power for Campgrounds",
}
path_to_model = "../Trail Power.aird"
model = capellambse.MelodyModel(path_to_model, resources=resources)
module_name = "Fred"
try :
    module = model.sa.requirement_modules.by_name(module_name)
except:
  print("Module :", modulename," not Found")



In [2]:
#String processing does like objects of type None. This is case when you pull content expecting to be a string and None is found.
def convert_nonetype (instr):
    if instr is None :
        instr = ""
        return instr
    else:
        #this fixes any encoding errors. 
        instr_bytes = instr.encode('utf-8')
        instr_str = instr_bytes.decode('utf-8')
        return instr_str

def sortnumber(outline_number):
    temp = outline_number.replace('-','.')
    ischapter = temp == outline_number 
    if ischapter :

        listofnumbers = temp.split('.')
        outlineforsort = ''
        for i in range(len(listofnumbers)) :
            listofnumbers[i] = listofnumbers[i].zfill(10)
        for temp in listofnumbers :
            outlineforsort = outlineforsort + temp + '.' 
        #pad for up to 10 header letters  .0000000000   
        #01.00.00.00.00.00
        #01.00.00.00.00.00
        #01.01.00.00.00.00
        # find out number of headers
        # add padding for up to 20 heading level
        levels = outlineforsort.count('.')
        padded_levels = 20 - levels
        for i in range(padded_levels):
            outlineforsort = outlineforsort + "0000000000."    
    else :
        #split into to parts x.x.x-x.x.x  first part is outline second is requirements place 
        outline_requirement = outline_number.split('-')
        print( outline_requirement[0])
        print( outline_requirement[1])
        listofnumbers = outline_requirement[0].split('.')
        outlineforsort = ''
        for i in range(len(listofnumbers)) :
            listofnumbers[i] = listofnumbers[i].zfill(10)
        for temp in listofnumbers :
            outlineforsort = outlineforsort + temp + '.'
        #01.00.00.00.00.00
        #01.00.00.R0.00.00   
        #01.00.00.R0.R1.00
        #01.00.00.R0.R2.00
        #01.00.00.R1.R0.00
        #01.01.00.00.00.00
        # find out number of headers
        # add padding for up to 10 heading levels for chapters
        levels = outlineforsort.count('.')
        padded_levels = 10 - levels
        for i in range(padded_levels):
            outlineforsort = outlineforsort + "0000000000."  
            
                
        listofnumbers = outline_requirement[1].split('.')
        req_outlineforsort = ''
        for i in range(len(listofnumbers)) :
            listofnumbers[i] = 'R' + listofnumbers[i].zfill(9)
        #concat the to parts
        for temp in listofnumbers :
            req_outlineforsort =  req_outlineforsort + temp + '.'
        #01.00.00.00.00.00
        #01.00.00.R0.00.00
        #01.00.00.R0.R1.00
        #01.00.00.R0.R2.00
        #01.00.00.R1.R0.00
        #01.01.00.00.00.00
        # find out number of headers for the requirement 
        # add padding for up to 10 heading level for requirement
        levels = req_outlineforsort.count('.')
        padded_levels = 10 - levels
        for i in range(padded_levels):
            req_outlineforsort = req_outlineforsort + "0000000000." 
        outlineforsort = outlineforsort +  req_outlineforsort
        
    return outlineforsort

def type_id(outline_number):
    temp = outline_number.replace('-','.')
    ischapter = temp == outline_number 
    if ischapter :
        this_type_id = "heading"    
    else:
        this_type_id = "systemrequirement"
    return this_type_id
    
def chapternumberplus1R(outline_number):
    temp = outline_number.replace('-','.')
    ischapter = temp == outline_number 
    if ischapter :
        listofnumbers = temp.split('.')
        outlineforsort = ''
        for i in range(len(listofnumbers)) :
            listofnumbers[i] = listofnumbers[i].zfill(10)
        for temp in listofnumbers :
            outlineforsort = outlineforsort + temp + '.'      
    else :
        #split into to parts x.x.x-x.x.x   first part is outline second is requirements 
        outline_requirement = outline_number.split('-')
        print( outline_requirement[0])
        print( outline_requirement[1])
        listofnumbers = outline_requirement[0].split('.')
        outlineforsort = ''
        for i in range(len(listofnumbers)) :
            listofnumbers[i] = listofnumbers[i].zfill(10)
        for temp in listofnumbers :
            outlineforsort = outlineforsort + temp + '.'
        listofnumbers = outline_requirement[1].split('.')
        req_outlineforsort = ''
        for i in range(len(listofnumbers)) :
            listofnumbers[i] = 'R' + listofnumbers[i].zfill(9)
            break;
        #concat the to parts
        for temp in listofnumbers :
            req_outlineforsort =  req_outlineforsort + temp + '.'
            break;
        outlineforsort = outlineforsort + req_outlineforsort
        print('level outline',outlineforsort)    
    return outlineforsort
  



def Populate_Parent_As_ID(moduleframe, row , level, parent) :
    #use a mix of while and do loop pattern, exiting on last row
    #while moduleframe.loc[row]['level'] >= level :
    while row < len(moduleframe.index)    and  moduleframe.loc[row]['level'] >= level :
        #print("Row Level", moduleframe.loc[row]['level'])
        if   moduleframe.loc[row]['level'] == level :
            print("row at same Level as previous")
            moduleframe.at[row ,"parent"]=parent
            #moduleframe.at[row ,"parent_row"]=parent_row
            #Print Line below may fail due to odd character in the fullname. 
            #print("Set row:",row,"Set Parent ID:",moduleframe.at[row ,"parent"],"Set id:",moduleframe.loc[row]['id'])  
        if  moduleframe.at[row,"level"] == level + 1 :
            #print("row at next level")
            new_parent = moduleframe.loc[row-1]['id'] 
            #new_parent_row = row-1
            new_level = moduleframe.loc[row]['level']
            row = Populate_Parent_As_ID(moduleframe, row , new_level , new_parent )# 
            #print("returned to previous level")
        #if  moduleframe.at[row,"level"] > level + 1 :
            #print("skipping row..",row)
            #print(moduleframe.loc[row]['Fullname'])
        row = row + 1
        if row == len(moduleframe.index) :
            break
    row = row - 1 
    return row    


def NumberinLevel(moduleframe, row , level, numberinlevel) :
    # sets the number with in a level 
    #use a mix of while and do loop pattern, exiting on last row
    #while moduleframe.loc[row]['level'] >= level :
    while row < len(moduleframe.index)    and  moduleframe.loc[row]['level'] >= level :
        #print("row", row,"id", moduleframe.loc[row]['id'], row ,"Row Level", moduleframe.loc[row]['level'])
        if   moduleframe.loc[row]['level'] == level :
            
            moduleframe.at[row ,"numberinlevel"] = numberinlevel
            print("row", row,"id", moduleframe.loc[row]['id'], row ,"Row Level", moduleframe.loc[row]['level'],"numberinlevel:",numberinlevel)
            numberinlevel = 1 + numberinlevel
            #moduleframe.at[row ,"parent_row"]=parent_row
            #Print Line below may fail due to odd character in the fullname. 
            #print("Set row:",row,"Set Parent ID:",moduleframe.at[row ,"parent"],"Set id:",moduleframe.loc[row]['id'])  
        if  moduleframe.at[row,"level"] == level + 1 :
             
            #print("row at next level")
            #new_parent_row = row-1
            new_level = moduleframe.loc[row]['level']
            print("Calling recursively setting new level",new_level)
            row = NumberinLevel(moduleframe, row , new_level , 0 )# 
            #print("returned to previous level")
        #if  moduleframe.at[row,"level"] > level + 1 :
            #print("skipping row..",row)
            #print(moduleframe.loc[row]['Fullname'])
        row = row + 1
        if row == len(moduleframe.index) :
            break
    row = row - 1 
    return row    

In [3]:
from pprint import pprint 
from lxml import etree
def Import_Requirements_from_ReqIF(file_name):
    def pp(e):
        print (etree.tostring(e, pretty_print=True))
    
    
    
    # Load and parse the REQIF XML file and the schema file
    reqif_file_path = 'TestTemplate_2.reqif'
    
    
    
    tree = etree.parse(reqif_file_path)
    root = tree.getroot()
    
    # Print the root element
    print(f"Root element: {root.tag}")
    
    # Define the namespaces (update if needed based on your REQIF file)
    namespaces = {
        'default': 'http://www.omg.org/spec/ReqIF/20110401/reqif.xsd',
        'ns2': 'http://www.w3.org/1999/xhtml'
    }
    
    
    print("namespaces:",namespaces)
    
    # Extract CORE-CONTENT using namespaces if present
    core_content = root.find('.//default:CORE-CONTENT', namespaces)
    if core_content is not None:
    
        Spec_Type_df = pd.DataFrame(
             {
            'identifier': [],
            'lastchange': [], 
            'longname': [],
            }) 
        
    
        spec_types =  core_content.findall('.//default:SPECIFICATION-TYPE', namespaces)
        for spec_type in  spec_types:
            obj_id = spec_type.get('IDENTIFIER', 'N/A')
            obj_last_change = spec_type.get('LAST-CHANGE', 'N/A')
            obj_long_name= spec_type.get('LONG-NAME', 'N/A')
            new_row = {'identifier' : obj_id ,
                       'lastchange' : obj_last_change,
                       'longname' : obj_long_name } 
            #print(new_row)
            Spec_Type_df.loc[len(Spec_Type_df)] = new_row
            
        display(Spec_Type_df)
    
        Spec_Object_Type_df = pd.DataFrame(
            {
            'identifier': [],
            'lastchange': [], 
            'longname': [],
            }) 
        spec_object_types =  core_content.findall('.//default:SPEC-OBJECT-TYPE', namespaces)
        for spec_object_type in  spec_object_types:
            obj_id = spec_object_type.get('IDENTIFIER', 'N/A')
            obj_last_change = spec_object_type.get('LAST-CHANGE', 'N/A')
            obj_long_name= spec_object_type.get('LONG-NAME', 'N/A')
            new_row = {'identifier' : obj_id ,
                       'lastchange' : obj_last_change,
                       'longname' : obj_long_name} 
            #print(new_row)
            Spec_Object_Type_df.loc[len( Spec_Object_Type_df)] = new_row
            
        display(Spec_Object_Type_df)
    
    
        Req_Spec = pd.DataFrame(
             {
            'outlineNumber': [],
            'level': [], 
            'sortNumber': [],
            'last update': [], 
            'id': [], 
            'type_id': [],
            'Fullname': [],
            'description': [] ,
            'parent': [] 
            })    
        
        spec_objects = core_content.findall('.//default:SPEC-OBJECT', namespaces)
    
        print("Requirements:")
    
        for spec_object in spec_objects:
            #pprint(spec_object)
            obj_id = spec_object.get('IDENTIFIER', 'N/A')
            #long_name_elem = spec_object.find('.//default:LONG-NAME', namespaces)
            long_name_elem = spec_object.get('LONG-NAME', 'N/A')
            description_elem = spec_object.find('.//default:DESC', namespaces)
            last_change_elem = spec_object.get('LAST-CHANGE', 'N/A')
            #long_name = long_name_elem.text if long_name_elem is not None else 'N/A'
            description = description_elem.text if description_elem is not None else 'N/A'
    
            print(f"- ID: {obj_id}")
            print(f"  Long Name: {long_name_elem}")
            print(f"  Last Change: {last_change_elem}")
        
            values = spec_object.findall('.//default:VALUES', namespaces)
            for value in values:
                #pp(value)
                htmls = value.findall('.//default:ATTRIBUTE-VALUE-XHTML', namespaces)
                for html in htmls:
                    #pp(html)
                    html_values = value.findall('.//default:THE-VALUE', namespaces)
                    for html_value in html_values:
                        #pp(html_value)
                        desc_element = html_value.find('.//ns2:div', namespaces)
                        if desc_element is not None:
                            print(f"  HTML Text: {desc_element.text}")
                        else:
                            print("  HTML Text element not found.")
                            desc_element = ""
                         
                section_numbers = value.findall('.//default:ATTRIBUTE-VALUE-STRING', namespaces)
                for section_number in section_numbers:  
                    section_number_element = section_number.get('THE-VALUE', 'N/A')
                print(f"  Section Number: {section_number_element}")   
                 
            #spec_type_identifiers = spec_object.findall('.//default:TYPE', namespaces)
            #for spec_type_identifier in spec_type_identifiers:  
            # print("spec type identifier:",spec_type_identifier.find('SPEC-OBJECT-TYPE-REF'))
            
            
            ##print the etree
            #pp(spec_object)
            #spec_object_type_ref = spec_object.find('.//SPEC-OBJECT/SPEC-OBJECT-TYPE-REF')
            spec_object_type_ref = spec_object.find('.//default:SPEC-OBJECT-TYPE-REF', namespaces)
            #pp(spec_object_type_ref)
            
            # Check if the element is found and extract the text
            if spec_object_type_ref is not None:
                mdreqif_value = spec_object_type_ref.text
                print(f"  Extracted value: {mdreqif_value}")
            else:
                print("  SPEC-OBJECT-TYPE-REF tag not found.")
            print()     
                
            new_row = {'outlineNumber' : section_number_element ,
                       'sortNumber' : sortnumber("0" ) ,
                       'Fullname' : long_name_elem,
                       'type_id': Spec_Object_Type_df.loc[Spec_Object_Type_df['identifier'] ==  mdreqif_value, 'longname'].values[0],
                       'description': desc_element.text,
                       'last update': last_change_elem ,
                       'id':  obj_id,
                       'level': chapternumberplus1R(convert_nonetype(section_number_element)).count('.') ,
                       'parent': "TBD" } 
            #print(new_row)
            Req_Spec.loc[len(Req_Spec)] = new_row      
    
    else:
        print("No CORE-CONTENT found in the REQIF file.")
    return Req_Spec
              


In [4]:
from openpyxl import *   
def Import_Requirements_from_Excel(file_name,sheet_name):

    exceldf = pd.read_excel(file_name,sheet_name)

    
    #print(exceldf.to_string(index=True))
          
    Req_Spec = pd.DataFrame(
         {
        'outlineNumber': [],
        'level': [], 
        'sortNumber': [],
        'last update': [], 
        'id': [], 
        'type_id': [],
        'Fullname': [],
        'description': [],
        'parent' : []
        })   
    
    for index, row in exceldf.iterrows():
        #print("Excel ID" , exceldf.loc[index]['Requirement ID'])
        #looks for id in SMW content
        full_name = '[' + exceldf.loc[index]['Requirement ID']+ ']' + ' '  + exceldf.loc[index]['Name'] 
        new_row = {'outlineNumber' : exceldf.loc[index]['Outline Number'] ,
                   'sortNumber' : sortnumber(exceldf.loc[index]['Outline Number'] ) ,
                   'Fullname' : full_name  , 'title' : exceldf.loc[index]['Name'],
                   'type_id': type_id(exceldf.loc[index]["Type"]),
                   'description': exceldf.loc[index]['Description'],
                   'last update': exceldf.loc[index]["Last Update"],
                   'id': exceldf.loc[index]['Requirement ID'],
                   'level': chapternumberplus1R(convert_nonetype(exceldf.loc[index]['Outline Number'])).count('.'),
                   'parent': "TBD" }  
        #print(new_row)
        Req_Spec.loc[len(Req_Spec)] = new_row
              
    
    #Pol_Req_Spec = pd.DataFrame(list(map(extract_attrs_of_interest, work_items)))
    return Req_Spec

In [5]:

Req_Spec = Import_Requirements_from_ReqIF('TestTemplate_2.reqif')
display(Req_Spec)

#Req_Spec = Import_Requirements_from_Excel("Requirements.xlsx","Sheet1")
#display(Req_Spec)

Root element: {http://www.omg.org/spec/ReqIF/20110401/reqif.xsd}REQ-IF
namespaces: {'default': 'http://www.omg.org/spec/ReqIF/20110401/reqif.xsd', 'ns2': 'http://www.w3.org/1999/xhtml'}


,identifier,lastchange,longname
0,MDReqIF_1725555825299_325506_1,2024-09-05T10:03:45,Package


,identifier,lastchange,longname
0,MDReqIF_1725555825518_364225_5,2024-09-05T10:03:45,ReqIF specific
1,MDReqIF_1725555825575_82330_13,2024-09-05T10:03:45,lamRequirement


Requirements:
- ID: _2022x_2_c0402fd_1725554288357_396916_18144
  Long Name: Requirements
  Last Change: 2024-09-05T10:03:45
  HTML Text:  Requirements
  Section Number: 5
  Extracted value: MDReqIF_1725555825518_364225_5

- ID: _2022x_2_c0402fd_1725554288997_760343_18626
  Long Name: Functional Requirements
  Last Change: 2024-09-05T10:03:45
  HTML Text:  Functional Requirements
  Section Number: 5.1
  Extracted value: MDReqIF_1725555825518_364225_5

- ID: _2022x_2_c0402fd_1725554288998_771525_18627
  Long Name: Performance Requirements
  Last Change: 2024-09-05T10:03:45
  HTML Text:  Performance Requirements
  Section Number: 5.2
  Extracted value: MDReqIF_1725555825518_364225_5

- ID: _2022x_2_c0402fd_1725554288998_554705_18628
  Long Name: Usability Requirements
  Last Change: 2024-09-05T10:03:45
  HTML Text:  Usability Requirements
  Section Number: 5.3
  Extracted value: MDReqIF_1725555825518_364225_5

- ID: _2022x_2_c0402fd_1725554289093_344828_18912
  Long Name: Measurable Sati

,outlineNumber,level,sortNumber,last update,id,type_id,Fullname,description,parent
0,5,1,0000000000.0000000000.0000000000.0000000000.00...,2024-09-05T10:03:45,_2022x_2_c0402fd_1725554288357_396916_18144,ReqIF specific,Requirements,Requirements,TBD
1,5.1,2,0000000000.0000000000.0000000000.0000000000.00...,2024-09-05T10:03:45,_2022x_2_c0402fd_1725554288997_760343_18626,ReqIF specific,Functional Requirements,Functional Requirements,TBD
2,5.2,2,0000000000.0000000000.0000000000.0000000000.00...,2024-09-05T10:03:45,_2022x_2_c0402fd_1725554288998_771525_18627,ReqIF specific,Performance Requirements,Performance Requirements,TBD
3,5.3,2,0000000000.0000000000.0000000000.0000000000.00...,2024-09-05T10:03:45,_2022x_2_c0402fd_1725554288998_554705_18628,ReqIF specific,Usability Requirements,Usability Requirements,TBD
4,5.3.3,3,0000000000.0000000000.0000000000.0000000000.00...,2024-09-05T10:03:45,_2022x_2_c0402fd_1725554289093_344828_18912,lamRequirement,Measurable Satisfaction Criteria,Measurable Satisfaction Criteria,TBD
5,5.3.2,3,0000000000.0000000000.0000000000.0000000000.00...,2024-09-05T10:03:45,_2022x_2_c0402fd_1725554289094_833986_18913,lamRequirement,Measurable Efficiency,Measurable Efficiency,TBD
6,5.3.1,3,0000000000.0000000000.0000000000.0000000000.00...,2024-09-05T10:03:45,_2022x_2_c0402fd_1725554289094_489872_18914,lamRequirement,Measurable Effectiveness,Measurable Effectiveness,TBD
7,5.4,2,0000000000.0000000000.0000000000.0000000000.00...,2024-09-05T10:03:45,_2022x_2_c0402fd_1725554288998_966051_18629,lamRequirement,Interface Requirements,Interface Requirements,TBD
8,5.4.5,3,0000000000.0000000000.0000000000.0000000000.00...,2024-09-05T10:03:45,_2022x_2_c0402fd_1725554289094_940638_18917,lamRequirement,Physical Connections,Physical Connections,TBD
9,5.4.4,3,0000000000.0000000000.0000000000.0000000000.00...,2024-09-05T10:03:45,_2022x_2_c0402fd_1725554289095_819837_18918,lamRequirement,Human Interface,Human Interface,TBD


In [11]:
Req_Spec.to_excel ( 'export_Req_Specspec1.xlsx', index = False, header=True)  
Req_Spec = Req_Spec.sort_values(by='sortNumber')
Req_Spec = Req_Spec.reset_index(drop=True)  # make sure indexes pair with number of rows
Req_Spec = Req_Spec.sort_values(by='sortNumber')
Req_Spec = Req_Spec.reset_index(drop=True)  # make sure indexes pair with number of row
row = 0
level = 1
xlsx_file_name = 'export_Req_Specspec2.xlsx'
print("Excel for Req_Spec Spec post parent processing",xlsx_file_name)
Req_Spec.to_excel ( xlsx_file_name, index = False, header=True)  

row = Populate_Parent_As_ID(Req_Spec, row , level, "Spec")

#check the dataframe before processing

xlsx_file_name = 'export_Req_Specspec3.xlsx'
print("Excel for Req_Spec Spec post parent processing",xlsx_file_name)
Req_Spec.to_excel ( xlsx_file_name, index = False, header=True)  


def Populate_ModuleObject(Req_Spec_DocItem,moduleObject):
    temp_string = convert_nonetype(Req_Spec_DocItem['id']) + ": " + convert_nonetype(Req_Spec_DocItem['title'])
    moduleObject.set_name(temp_string)
    moduleObject.set_prefix(Req_Spec_DocItem['id'])
    temp_string = convert_nonetype(Req_Spec_DocItem['outlineNumber']) + " " + convert_nonetype(Req_Spec_DocItem['id']) + " " + convert_nonetype(Req_Spec_DocItem['Fullname'])
    moduleObject.set_long_name(temp_string)
    temp_string_1 = '<p>'+convert_nonetype(Req_Spec_DocItem['id']) + ": " + convert_nonetype(Req_Spec_DocItem['title'] )+'</p>'
    temp_string_2 = '<p>Updated: '+ convert_nonetype(Req_Spec_DocItem['updated']) + '</p>'
    temp_string_3 = '<p>'+ convert_nonetype(Req_Spec_DocItem['description']) + '</p>'
    temp_string = temp_string_1  + temp_string_2 + temp_string_3
    moduleObject.set_text(temp_string)
    return moduleObject


def create_module_item(Req_Spec_DocItem, parent_module):
    #this code is dependent on the python library capellambse
    model.start_transaction()
    try:
        #create requitement sibling
        #print("Creating module item Req_Spec Type:",Req_Spec_DocItem['type_id'])
        if Req_Spec_DocItem['type_id'] == "heading" :
            moduleObject = Folder()
            #print("Creating Folder")
            moduleObject = Populate_ModuleObject(Req_Spec_DocItem,moduleObject)
            
            parent_module.get_owned_requirements().add(moduleObject)
        else:
            moduleObject = Requirement()
            moduleObject = Populate_ModuleObject(Req_Spec_DocItem,moduleObject)
            parent_module.get_owned_requirements().add(moduleObject)
                
    except:
        # if something went wrong we rollback the transaction
        model.rollback_transaction()
        raise
    else:
        # if everything is ok we commit the transaction
        model.commit_transaction()

    
    # save the Capella model
    model.save()
    return moduleObject

print(module) 
help(model)
if module.requirements == [] :
    
    #start creation of content in the model
    
    
    try:
        #change name of Module to name of Req_Spec Document
        module.long_name = Req_Spec.loc[0]['Fullname']
       
                
    except:
        # if something went wrong we rollback the transaction
        raise
    else:
        # if everything is ok we commit the transaction
         model.save()
        #print("Model Committed")
    
    # save the Capella model
    #print("Module Name;",module.get_name())
    #delete the row with document information        

        
    

    
    #Funtion to create a structure in Capella/SMW when nothing exits previously 
    # it returns a row in table to support recursion
    def create_requirement_module_object(Source_Req_DF, moduleparent , row, current_level):
        """
        Populates a SMW module with a requirement from a row in the dataframe 
        
        Args:
                Source_Req_Dc : Panda data frame with rows to be populated.  
                moduleparent : The is the parent that requirement will be added as a child.
                row : this is the row in  Req_Spec being processed.
                current_level : This is the level in structure that requirement are being created.
                
        #this is a recursive algorithm designed to produce content in document level
        #the procedure invoke itself when it detects indention change in document
        #it will loop at a level while the document ends and greater than equal to when called.
        #use a modified do loop pattern
        """
        new_module_parent = moduleparent
        while   row < len( Source_Req_DF.index)    and  Source_Req_DF.loc[row]['level'] >= current_level :
            Source_Req_DF_Row = Source_Req_DF.loc[row]
            #print("Row:",row)
            print('Source Item ID', Source_Req_DF_Row['id'])
            print('Source Item Outline',Source_Req_DF_Row['outlineNumber'])
            Source_Req_DF_Row_level = Source_Req_DF_Row['level']
            print('Source Item Level', Source_Req_DF_Row_level) 
            print('current_level', current_level) 
            if Source_Req_DF_Row_level == current_level: 
                print("creating new module object")
                new_module_parent = create_module_item(Source_Req_DF_Row, moduleparent )
                
            if Source_Req_DF_Row_level == current_level + 1: 
                # since spec is indenting a one level  
                print("switched to new parent module",new_module_parent)
                print("new parent  name",new_module_parent.get_name)
                row =  create_requirement_module_object(Source_Req_DF,  new_module_parent  , row, Source_Req_DF_Row_level)
            row = row + 1
            if row == len(Source_Req_DF.index) :
                break
            if Source_Req_DF_Row_level > current_level + 1:
                #this code may not be needed. It would require a Req_Spec document to skip a level of indention to encoutner it. It could be removed if deemed impossible.
                #print("Skipping row:", row ,"Title:" ,Req_Spec_DocItem['title'], "due to unexpected change to a indention of to",Req_Spec_DocItem_level)
                #it will be reflected as a singel level
                row =  create_requirement_module_object(Source_Req_DF,  new_module_parent  , row, Source_Req_DF_Row_level)
                row = row + 1;
        # go back a row since it was incremented and return 
        row = row - 1       
        return row



    
    #Processing inports... 
    starting_level = 0
    starting_row = 0
    #print("Number of Rows", len(Req_Spec.index))
    row = create_requirement_module_object(Req_Spec, module , starting_row, starting_level)
    #print("Last row",row)
    
    ##


else:
    #Process against existing content.
    # Create data frame for SMW

    def get_suspect (longName):
        #print('Suspect Check of longname:',longName[0:9])
        if longName[0:9]== 'Suspect: ':
            print("is suspect")
            return 'Suspect:'
        else :    
            return ''
        
    def get_deleted (longName):
        #print('Suspect Check of longname:',longName[0:9])
        if longName[0:9]== 'Deleted: ':
            print("is deleted")
            return 'Deleted:'
        else :    
            return ''

    def parse_outline_number (longName):
        try :
            if get_suspect (longName) == '' :
                #print("parsing outline suspect")
                parsedString = longName.split(' ')
                #print(parsedString)
                #print(parsedString[0])
                return parsedString[0]
            elif get_deleted (longName) == '' :
                #print("parsing outline suspect")
                parsedString = longName.split(' ')
                #print(parsedString)
                #print(parsedString[0])
                return parsedString[0]
            else :
                parsedString = longName.split(' ')
                #print(parsedString)
                #print(parsedString[1])
                return parsedString[1]
                
        except :
            #added for case where string may not be parseable
            return "Not Parsed"
        
    
    def convert_to_type_id(moduleelementtype):
        if moduleelementtype == "Folder" :
            return "heading"
        else :
            return "requirement" 
        
        
    def convert_nonetype (instr):
        if instr is None :
            instr = ""
            return instr
        else:
            return instr  
        
    def parse_text(text): 
        try:  
            text_list = text.split("Updated: ",1)
            text = text_list[1]
            print('text:',text)
            text_list = text.split("</p>",1)
            text = text_list[0]
            print('text:',text)
            return text
        except :
            #added for case where string may not be parseable
            return ''

    def extract_smw_attrs_of_interest(modElement):
        return dict(
            outlineNumber=parse_outline_number(modElement.get_long_name()),
            sortNumber=sortnumber(parse_outline_number(modElement.get_long_name())),
            #outlineNumber= modElement.get_long_name(),
            Fullname=modElement.get_name(),
            type_id= convert_to_type_id(type(modElement).__name__),
            description=convert_nonetype(modElement.get_text()),
            updated=parse_text(convert_nonetype(modElement.get_text())),
#            level=parse_outline_number(modElement.get_long_name()).count('.'),
            id=modElement.get_prefix(),
            suspect=get_suspect(modElement.get_long_name()),
            #in_links=modElement.get_incoming_linked_elems(),
            #out_links=modElement.get_outgoing_linked_elems()
            element=modElement,
            incominglinks=modElement.get_incoming_linked_elems(),
            action = 'none'  #this will be used to denote what may be required to model, add, delete, update outline, updated workitem
            
            )
        
    modElements = module.get_all_contents()    
    moduleframe = pd.DataFrame(list(map(extract_smw_attrs_of_interest, modElements)))
    #print(moduleframe)

    print(moduleframe.to_string(index=True))

    for index, row in moduleframe.iterrows():

        # TODO ficx level computation
        moduleframe.at[index,"level"]=chapternumberplus1R(convert_nonetype(moduleframe.loc[index]['outlineNumber'])).count('.')
        
    
    row = 0
    level = 1
    #parent_row = -1 # -1 is the module
    #print("Populating Parents")
    #row = Populate_Parent(moduleframe, row , level, module)
    row = Populate_Parent_As_ID(moduleframe, row , level, "Spec")
    print('Last Row:', row)
   

#      For each in Req_Spec
#              Use Id to see if it is in SMW
#               If it is then check if needs updating
#              If not add into SMW table at point where higher than existing 
#               update the parent object with the elements at the same level.
# Level plus I’d
    Requirement_Adds = 0
    Requirement_Deletes = 0
    Requirement_Outline_Updates = 0
    Requirement_Updates = 0
    moduleframe = moduleframe.reset_index(drop=True)  # make sure indexes pair with number of rows
    priorid = "module" 
    for index, row in Req_Spec.iterrows():
        print("Req_Spec ID" , row['id'])
        #looks for id in SMW content
        modulerequirementframe = moduleframe.loc[moduleframe['id'].str.contains(row['id'])]   
        #print( modulerequirementframe)
        #print(modulerequirementframe.loc[0]['title'])
        if modulerequirementframe.empty : #this means it did not find anything
            print("Missing Requirement",row['id'])
            print("going to insert after",priorid)
            #print("Req_Spec row level",row['level'])
            priorrow = moduleframe.loc[moduleframe['id'].str.contains(priorid)]
            priorrow = priorrow.reset_index(drop=True)  # make sure indexes pair with number of rows
            print("prior row",priorrow)                           
            print("prior module requirement level", priorrow.at[0,'level'])
            #old way depreciated but worked
            new_row = {'outlineNumber' : Req_Spec.loc[index]['outlineNumber']  ,
                       'sortNumber' : sortnumber(Req_Spec.loc[index]['outlineNumber'] ) , 
                       'Fullname' : Req_Spec.loc[index]['Fullname'] ,
                       'type_id': Req_Spec.loc[index]['type_id'] ,
                       'description': ' ',
                       'updated': ' ','id': Req_Spec.loc[index]['id'],
                       'element': '', 'incominglinks': [] ,
                       'action': 'add' ,
                       'level': Req_Spec.loc[index]['level'],
                       'parent': Req_Spec.loc[index]['parent']} 
            moduleframe= moduleframe.append(new_row, ignore_index=True)
            #pd.concat([moduleframe,pd.DataFrame(new_row.values(), columns=moduleframe.columns)], ignore_index=True)
            # new was 
            #new_row = pd.Series( {'outlineNumber' : Req_Spec.loc[index]['outlineNumber']  , 'Fullname' : Req_Spec.loc[index]['Fullname'] , 'type_id': Req_Spec.loc[index]['type_id'] ,'description': ' ', 'updated': ' ','id': Req_Spec.loc[index]['id'], 'element': '', 'action': 'add' ,'level': Req_Spec.loc[index]['level'], 'parent': Req_Spec.loc[index]['parent']})
            #pd.concat([moduleframe, new_row.to_frame().T], ignore_index=True)
            Requirement_Adds = Requirement_Adds + 1
        else:
            modulerequirementframe = modulerequirementframe.reset_index() 
            #print( modulerequirementframe)  
            #print( 'module outline', modulerequirementframe.loc[0]['outlineNumber'],"module updated :",modulerequirementframe.loc[0]['updated'],':') 
            #print( 'Pol Spec outline number', Req_Spec.loc[index]['outlineNumber'],'Pol Spec updated :', Req_Spec.loc[index]['updated'],':') 
            if Req_Spec.loc[index]['updated'] != modulerequirementframe.loc[0]['updated'] :
                moduleframe.loc[moduleframe['id'].str.contains(row['id']),'action'] = 'update'
                #print("Needs full update")
                Requirement_Updates = Requirement_Updates + 1
            else:
                if modulerequirementframe.loc[0]['outlineNumber'] != Req_Spec.loc[index]['outlineNumber'] :
                    #print("Needs outline update")
                    moduleframe.loc[moduleframe['id'].str.contains(row['id']),'action'] = 'update_outline'
                    Requirement_Outline_Updates + 1
            #print( 'module outline number', modulerequirementframe.loc[0]['outlineNumber']) 
            #print( 'Pol Spec outline number', Req_Spec.loc[index]['outlineNumber']) 
            #if level is the same 0
                #insert new row and tag for add 
                #tag the parent row for update
            #   
            #
            #if level is higher make the prior a parent and set new hobject as parent. 
            
        priorid = row['id']
    
    # for each module requirement 
    # check to see if in the Req_Spec Spec
    # If it is not there mark the modelule Requirement for delection.
    priorid = "module" 
    for index, row in moduleframe.iterrows():
        #print("Module ID" , row['id'])
        #looks for id in SMW content
        Req_Specframe = Req_Spec.loc[Req_Spec['id'].str.contains(row['id'])]   
        #print( modulerequirementframe)
        #print(modulerequirementframe.loc[0]['title'])
        if  Req_Specframe.empty : #this means it did not find anything
            #print("Missing Requirement in Req_Spec",row['id'])
            #mark the requirement for deletion
            moduleframe.loc[moduleframe['id'].str.contains(row['id']),'action'] = 'delete'
            #force it at bottom when sorted
            moduleframe.loc[moduleframe['id'].str.contains(row['id']),'outlineNumber'] = 'Deleted'
            Requirement_Deletes = Requirement_Deletes + 1          
        priorid = row['id']    
        
    print("Inserted Requirements:",Requirement_Adds)
    print("Updated Requirements:",Requirement_Updates)
    print("Updated Requirement Outline Number:",Requirement_Outline_Updates)
    print("Deleted Requirement:",Requirement_Deletes)
    
    #resort
    moduleframe = moduleframe.sort_values(by='sortNumber')
    #print(Req_Spec)
    moduleframe = moduleframe.reset_index(drop=True)  # make sure indexes pair with number of rows
    
    row = 0
    level = 1
    row = NumberinLevel(moduleframe, row , level, 0)  
    
    project = CapellaPlatform.getProject(project_name)
    folder = CapellaPlatform.getFolder(project,'Reports')
    CapellaPlatform.getAbsolutePath(folder)
    
    xlsx_file_name = CapellaPlatform.getAbsolutePath(folder) + '/' + 'export_moduleframe1.xlsx'
    print("Excel file for Module",xlsx_file_name)   
    moduleframe.to_excel ( xlsx_file_name, index = False, header=True)
    
    
    xlsx_file_name = CapellaPlatform.getAbsolutePath(folder) + '/' + 'export_Req_Specspec4.xlsx'
    print("Excel for Req_Spec Spec",xlsx_file_name)
    Req_Spec.to_excel ( xlsx_file_name, index = False, header=True)  
    
    def req_index(parent,child):
        count = 0
        for req in parentObject.get_owned_requirements():
            if req == child :
                break
            count = count +1 
        return count     
    
    for index, row in moduleframe.iterrows():
        print("Action" , row['action'])
        #looks for id in SMW content
        if row['action'] == 'delete' :
            print('Processing deletes',row['id'])
            print('Object to mark deleted',row["element"])
            requirement = row['element']
            row['Fullname'] = "Deleted:" +  requirement.get_long_name()
            model.start_transaction()          
            try:
                requirement.set_name(row['Fullname'])
                requirement.set_long_name(row['Fullname'])
            except:
                # if something went wrong we rollback the transaction
                print('Error:Exception handled during setting requirement', row['id'], "to Deleted")
                model.rollback_transaction()
                raise
            else:
                # if everything is ok we commit the transaction
                model.commit_transaction()
                print("Model Committed")
            
            # save the Capella model
            model.save()   
        if row['action'] == 'update_outline' :
            print('Processing updates outline',row['id'])
            Req_Specframe = Req_Spec.loc[Req_Spec['id'].str.contains(row['id'])] 
            Req_Specframe = Req_Specframe.reset_index() 
            new_long_name = Req_Specframe.loc[0]['outlineNumber'] + ' ' + row['Fullname']
            model.start_transaction()          
            try:
                print('Object to update outline',row["element"])
                requirement = row['element']
                requirement.set_long_name(new_long_name)
            except:
                # if something went wrong we rollback the transaction
                print('Error:Exception handled updated Outline Number of', row['id'])
                model.rollback_transaction()
                raise
            else:
                # if everything is ok we commit the transaction
                model.commit_transaction()
                print("Model Committed")
            
            # save the Capella model
            model.save()            
        if row['action'] == 'update' :
            print('Processing updates',row['id'])
            Req_Specframe = Req_Spec.loc[Req_Spec['id'].str.contains(row['id'])] 
            Req_Specframe = Req_Specframe.reset_index() 
            print('Are there links for suspect',moduleframe.loc[index]['incominglinks'])
            if moduleframe.loc[index]['incominglinks'] != [] :
                print("Setting Suspect")
                new_long_name = 'Suspect: ' + Req_Specframe.loc[0]['outlineNumber'] + ' ' + row['Fullname']
            else :
                new_long_name = Req_Specframe.loc[0]['outlineNumber'] + ' ' + row['Fullname']
            model.start_transaction()          
            try:
                print('Object to update',row["element"])
                requirement = row['element']
                requirement = Populate_ModuleObject(Req_Specframe.loc[0],requirement)
                #mark the requirement suspect
                requirement.set_long_name(new_long_name)
            except:
                # if something went wrong we rollback the transaction
                print('Error:Exception handled updating text', row['id'])
                model.rollback_transaction()
                raise
            else:
                # if everything is ok we commit the transaction
                model.commit_transaction()
                print("Model Committed")
            
            # save the Capella model
            model.save()            
        if row['action'] == 'add' :
            print('Processing adds to create a new requirement and append to owner',row['id'])
            #create the new element fiest
            Req_Specframe = Req_Spec.loc[Req_Spec['id'].str.contains(row['id'])] 
            Req_Specframe = Req_Specframe.reset_index() 
            model.start_transaction()
            try:
                #create requitement sibling
                print("Req_Spec Type:",Req_Specframe.loc[0]['type_id'],"Adding Req_Spec ID:",Req_Specframe.loc[0]['id'])
                if Req_Specframe.loc[0]['type_id'] == "heading" :
                    moduleObject = Folder()
                    print("Creating Folder")
                    moduleObject = Populate_ModuleObject(Req_Specframe.loc[0],moduleObject)
                else:
                    print("Creating Requirement")
                    moduleObject = Requirement()
                    moduleObject = Populate_ModuleObject(Pol_Req_Specframe.loc[0],moduleObject)
                    
                 
            except:
                # if something went wrong we rollback the transaction
                model.rollback_transaction()
                raise
            else:
                # if everything is ok we commit the transaction
                model.commit_transaction()
 
            
            # save the Capella model
            model.save()    
            
            #get the parent from the moduleframe         
          
            #row['element'] = moduleObject
            #moduleframe.loc[moduleframe['id'].str.contains(row['id']),'outlineNumber'] = 'Deleted'
            moduleframe.loc[index,'element'] = moduleObject
            #print('moduleObject:',moduleframe.loc[index]['element'])
            
            
            #get the object for the parent 
            #Putting object in correct location
            parentID = row['parent']
            if parentID == "Spec" :
                parentObject = module
                #print('Parent Object is the Module')
            else :
                #going to find first row whose ID matches that of parent. 
                modulerequirementframe = moduleframe.loc[moduleframe['id'].str.contains(row['parent'])] 
                #print('modulerequirementframe',modulerequirementframe) 
                modulerequirementframe = modulerequirementframe.reset_index(drop=True) 
                #print('modulerequirementframe reindexed ',modulerequirementframe) 
                parentObject =  modulerequirementframe.loc[0]['element']
                #print('Parent Object',modulerequirementframe.loc[0]['id'])
                
            print('Owned Requirement',parentObject.get_owned_requirements())
            print("add new reauirement at end of list")
            model.start_transaction()          
            try:
                # had to use a java api  
                #parentObject.get_owned_requirements.get_java_object().add(insert_location,moduleObject)
                print("owned requirement via java",parentObject.get_java_object().getOwnedRequirements()) 
                print("Size owned requirement via java",parentObject.get_java_object().getOwnedRequirements().size()) 
                #parentObject.get_java_object().getOwnedRequirements().add(insert_location,moduleObject)#  error AttributeError: 'Folder' object has no attribute '_get_object_id'
                #add to the list of the owned requirements. 
                parentObject.get_owned_requirements().add(moduleObject)
                print("Requirement order after add")
                for req in parentObject.get_owned_requirements():
                    print("Req ID", req.get_prefix(),"current position",req_index(parentObject,req))
                print("Size owned requirement via java after adding",parentObject.get_java_object().getOwnedRequirements().size()) 
                #move from the last object to the correct location with java
                
                #for req in parentObject.get_owned_requirements():
                #   print("Req ID",req.get_prefix())
            except:
                # if something went wrong we rollback the transaction
                print('Error:Could not add new requirement to parent ')
                model.rollback_transaction()
                raise
            else:
                # if everything is ok we commit the transaction
                model.commit_transaction()
                print("Model Committed")
            # save the Capella model
            model.save()
            

    
    for index, row in moduleframe.iterrows():
        #looks for id in SMW content
        #if row['action'] == 'add' :
        parentID = row['parent']
        if parentID == "Spec" :
            parentObject = module
            #print('Parent Object is the Module')
        else :
            #going to find first row whose ID matches that of parent. 
            modulerequirementframe = moduleframe.loc[moduleframe['id'].str.contains(row['parent'])] 
            #print('modulerequirementframe',modulerequirementframe) 
            modulerequirementframe = modulerequirementframe.reset_index(drop=True) 
            #print('modulerequirementframe reindexed ',modulerequirementframe) 
            parentObject =  modulerequirementframe.loc[0]['element']
            #print('Parent Object',modulerequirementframe.loc[0]['id'])
        
    
        #for each parent object of a ADD check the childer positions and move them to desired
        ##for req in parentObject.get_owned_requirements():
        req = row['element']
        current_position = req_index(parentObject,req)
        #find module row where element match the requirement 
        print('req:',req)
        print('element:',moduleframe['element'])
        modulerow = moduleframe.loc[moduleframe['id'].str.contains(req.get_prefix())] 
        print(modulerow.to_string(index=True))
        desired_position =  int(modulerow['numberinlevel'])
        print("Req ID",req.get_prefix(),"Current Position",current_position,"Desired Position",desired_position)
        
        if current_position != desired_position :
        #if position from dataframe does not equal the position
        #then move from position to desired position
            model.start_transaction()          
            try:
                print("Order Before") 
                for req in parentObject.get_owned_requirements():
                    print("Req ID", req.get_prefix())
                
                #move from the last object to the correct location with java
                #parentObject.get_java_object().getOwnedRequirements().move(row['numberinlevel'],parentObject.get_java_object().getOwnedRequirements().size()- 1)
                parentObject.get_java_object().getOwnedRequirements().move(desired_position,current_position)
                #print("Requirement order after move to insert location",insert_location)
                print("Order After") 
                for req in parentObject.get_owned_requirements():
                    print("Req ID",req.get_prefix())
            except:
                # if something went wrong we rollback the transaction
                print('Error:Exception handled during to move of children',row['id'] )
                model.rollback_transaction()
                raise
            else:
                # if everything is ok we commit the transaction
                model.commit_transaction()
                print("Model Committed")
            # save the Capella model
            model.save()
            print("Model Saved")
            
                
                
                
                
                
             
             
         
            
                               
              
    #TIme to check the matices.   
    
    xlsx_file_name = CapellaPlatform.getAbsolutePath(folder) + '/' + 'export_moduleframe2.xlsx'
    print("Excel file for Module",xlsx_file_name)
    moduleframe.to_excel ( xlsx_file_name, index = False, header=True)  
    print(moduleframe.to_string(index=True))

    

   
        
    
    

Excel for Req_Spec Spec post parent processing export_Req_Specspec2.xlsx
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previous
row at same Level as previo

AttributeError: 'CapellaModule' object has no attribute 'get_name'